# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_read = pd.read_csv("../WeatherPy/weather_df.csv")
weather_df = weather_read.drop(columns=["Unnamed: 0"])
weather_df_vac = weather_df.dropna(how="any")
weather_df_vac.head()

City  Cloudiness Country        Date  Humidity    Lat     Lng  \
0  alta gracia           0      AR  1589134328        31 -31.65  -64.43   
1         chuy          37      UY  1589134329        51 -33.70  -53.46   
2    caravelas          49      BR  1589134304        59 -17.71  -39.25   
3    busselton          80      AU  1589134331        70 -33.65  115.33   
4   corralillo         100      CU  1589134332        84  22.98  -80.59   

   Max Temp  Wind Speed  
0     79.00       16.11  
1     73.63       10.51  
2     76.46        9.98  
3     48.00        2.68  
4     77.22        2.98

In [3]:
weather_df_vac.count()

City          572
Cloudiness    572
Country       572
Date          572
Humidity      572
Lat           572
Lng           572
Max Temp      572
Wind Speed    572
dtype: int64

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key= g_key)

In [5]:
# Store latitude and longitude in locations
locations  = weather_df_vac[["Lat","Lng"]]

# Filla NaN values and convert to float
humidity = weather_df_vac["Humidity"].astype("float")

In [6]:
# Plot Heatmap
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(
    locations,
    weights = humidity,
    dissipating = False,
    max_intensity = 10,
    point_radius=1
)

fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
weather_df_vac.head()

City  Cloudiness Country        Date  Humidity    Lat     Lng  \
0  alta gracia           0      AR  1589134328        31 -31.65  -64.43   
1         chuy          37      UY  1589134329        51 -33.70  -53.46   
2    caravelas          49      BR  1589134304        59 -17.71  -39.25   
3    busselton          80      AU  1589134331        70 -33.65  115.33   
4   corralillo         100      CU  1589134332        84  22.98  -80.59   

   Max Temp  Wind Speed  
0     79.00       16.11  
1     73.63       10.51  
2     76.46        9.98  
3     48.00        2.68  
4     77.22        2.98

In [8]:
weather_df_vac.count()

City          572
Cloudiness    572
Country       572
Date          572
Humidity      572
Lat           572
Lng           572
Max Temp      572
Wind Speed    572
dtype: int64

In [9]:
# Conditions
# A max temperature lower than 80 degrees, but higher than 70
# Wind speed less than 10 mph
# Zero Cloudiness
# Drop any rows that dont contain all three conditions

# conditions_df = pd.DataFrame(weather_df_vac, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness"])

temp_cond = weather_df_vac.loc[(weather_df_vac["Max Temp"] <= 80) & (weather_df_vac["Max Temp"] > 70)]
wind_speed = temp_cond.loc[(temp_cond["Wind Speed"] < 10)]
cloudiness = wind_speed.loc[(wind_speed["Cloudiness"] == 0)]


cloudiness.head()


City  Cloudiness Country        Date  Humidity    Lat     Lng  \
36        castro           0      BR  1589134369        55 -24.79  -50.01   
196  east london           0      ZA  1589134396        39 -33.02   27.91   
203    abu kamal           0      SY  1589134634        27  34.45   40.92   
278    panzhihua           0      CN  1589134752        37  26.59  101.71   
284      sakakah           0      SA  1589134759        17  29.97   40.21   

     Max Temp  Wind Speed  
36      70.43        6.15  
196     74.14        5.44  
203     77.59        7.38  
278     72.52        1.21  
284     75.20        4.70

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
# Store into variable named hotel_df.
hotel_df = cloudiness
# Add a "Hotel Name" column to the DataFrame. 
# = "" to specify initial entry
hotel_df["Hotel Name"] = ""
hotel_df

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


City  Cloudiness Country        Date  Humidity    Lat     Lng  \
36          castro           0      BR  1589134369        55 -24.79  -50.01   
196    east london           0      ZA  1589134396        39 -33.02   27.91   
203      abu kamal           0      SY  1589134634        27  34.45   40.92   
278      panzhihua           0      CN  1589134752        37  26.59  101.71   
284        sakakah           0      SA  1589134759        17  29.97   40.21   
297      turkistan           0      KZ  1589134772        35  43.00   68.00   
353  chalchihuites           0      MX  1589134842        21  23.48 -103.88   
377       kirakira           0      SB  1589134890        81 -10.45  161.92   

     Max Temp  Wind Speed Hotel Name  
36      70.43        6.15             
196     74.14        5.44             
203     77.59        7.38             
278     72.52        1.21             
284     75.20        4.70             
297     73.99        8.95             
353     72.45        6.35             
377     78.48        7.45

In [11]:
len(hotel_df["City"])

8

In [12]:
# Create a list to store the hotels
hotels = []

# set up a parameters dictionary
params = {
    "keyword": "Hotel",
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate to find nearest hotels
# Looping with iterrows()
for index,row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Change location for each iteration with the same params
    params["location"]=f"{lat},{lng}"
    
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    
    # convert response to json
    hotel_data = response.json()
    
    # Use try-except in case data is missing.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


City  Cloudiness Country        Date  Humidity    Lat     Lng  \
36          castro           0      BR  1589134369        55 -24.79  -50.01   
196    east london           0      ZA  1589134396        39 -33.02   27.91   
203      abu kamal           0      SY  1589134634        27  34.45   40.92   
278      panzhihua           0      CN  1589134752        37  26.59  101.71   
284        sakakah           0      SA  1589134759        17  29.97   40.21   
297      turkistan           0      KZ  1589134772        35  43.00   68.00   
353  chalchihuites           0      MX  1589134842        21  23.48 -103.88   
377       kirakira           0      SB  1589134890        81 -10.45  161.92   

     Max Temp  Wind Speed                  Hotel Name  
36      70.43        6.15  Pousada Rota Dos Tropeiros  
196     74.14        5.44           Blue Lagoon Hotel  
203     77.59        7.38                              
278     72.52        1.21                  7 Days Inn  
284     75.20        4.70             Raoum Inn Hotel  
297     73.99        8.95                              
353     72.45        6.35              Hotel La Palma  
377     78.48        7.45

In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(locations)
# Customize the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

# Add the layer to the map
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
# Display Map
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…